<a href="https://colab.research.google.com/github/cdmurray80/object_detection/blob/main/factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
np.set_printoptions(precision=4)

In [ ]:

#
# Helper functions
#
# Given covariance, compute correlation
def correlation_from_covariance(covariance):
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    return correlation


In [ ]:
#
# CHANGE THIS AS YOU SEE FIT
#
# Create our covariance matrix
cov_diag_term = 0.5;
COV = np.matrix([[1.0,cov_diag_term],[cov_diag_term,1.0]]);


#Given covariance and loadings, compute variance.
#If loadings represent differences from target, this will return tracking variance
def compute_variance(loadings):
  return(np.matmul(loadings.transpose(),np.matmul(COV, loadings))[0,0])
def compute_variance_xy(x, y):
  return compute_variance(np.matrix([[x],[y]]))
#
# Sanity check, and show 
assert(cov_diag_term < (COV[0,0]*COV[1,1])**0.5)
cor = correlation_from_covariance(COV)
print('Covariance:\n', COV, '\nCorrelation:\n', cor)


In [ ]:
x, y = np.meshgrid(np.linspace(-5, 5, 50),
                   np.linspace(-5, 5, 50))
fig, ax = plt.subplots(1)
v_func = np.vectorize(compute_variance_xy)    # major key!
ax.contour(x, y, v_func(x, y), [1., 2., 3., 4., 5.])
ax.grid()
plt.title( 'Equi-variance contour plot')
plt.xlabel('Factor 1 Exposure')
plt.ylabel('Factor 2 Exposure')
plt.show()


In [ ]:
def arrow_w_title(plt_ax, x, y, title, color='b', head_length=0.1, head_width=0.1, x0=0, y0=0):
  plt_ax.arrow(x0,y0,x,y, color=color,head_length = head_length, head_width=head_width, length_includes_head=True)
  plt_ax.annotate(title, xy=(x,y+0.3))

#
# Plot vectors to show risk of unit exposure to either factor
ax = plt.axes()
unit_var = compute_variance(np.matrix([[1.],[1.]]))
arrow_w_title(ax, COV[0,0]**0.5,0, 'Factor 1')

# This is a bit subtle...we use the covariance to scale our plot so that the
# length of a vector with unit exposure to each factor is the risk of 
# unit exposure that each factor.
#factor_2_dx = (unit_var - cov[1,1])**0.5 - cov[0,0]**0.5
factor_2_dx = (unit_var - COV[0,0] - COV[1,1])/(2*(COV[0,0]**0.5))
factor_2_dy = (COV[1,1] - factor_2_dx**2)**0.5
arrow_w_title(ax, factor_2_dx,factor_2_dy, 'Factor 2')
arrow_w_title(ax, COV[0,0]**0.5 + factor_2_dx,factor_2_dy, 'Factor1 + Factor2', color='g')
ax.set_xlim(-1,6)
ax.set_ylim(-1,3)
plt.title('Unit exposure to each factor, as well as to both')
plt.show()

In [ ]:
print('Port 1 has  1 unit  of factor 1 and risk:',compute_variance(np.matrix([[1.],[0.]]))**0.5)
print('Port 2 has ~4 units of factor 2 and risk:',compute_variance( np.matrix([[0.],[4.08]]))**0.5)
print('Port 3 has  1 unit  of factor 2 and risk:',compute_variance( np.matrix([[0.],[1]]))**0.5)
print('Covariance\n',COV, '\nCovariance sqroooted\n', np.sqrt(COV))

In [ ]:
# Sanity check that the risk of unit exposure to each factor equals the plotted L2 norm
print('Risk: {:.5f}, L2 Norm: {:.5f}'.format( unit_var ** 0.5, ((COV[0,0]**0.5 + factor_2_dx)**2 + factor_2_dy**2)** 0.5))

In [ ]:
#Do eigen decomposition...annoying but we need extra code to sort eigenvectors so it's more than 1 line
# Probably 95% of users want the result sorted...idk why it's not an argument to return in sorted order
eigenvalues, eigenvectors = np.linalg.eig(COV)
idx = (eigenvalues).argsort()[::-1]
eigenvectors = eigenvectors[:,idx]
eigenvalues = eigenvalues[idx]

# Plot samples and eigenvectors
samples=np.random.multivariate_normal(np.array([0,0]),COV, size=4000)
ax=plt.axes()
plt.scatter(samples[:,0],samples[:,1])
ax.set_xlim(-6,6)
ax.set_ylim(-6,6)
for i in range(2):
  arrow_w_title(ax, eigenvectors[0,i]*eigenvalues[i]**0.5,eigenvectors[1,i]*eigenvalues[i]**0.5, 'EigVec ' + str(i+1), color='r')
  print( 'Eigenvalue ', i, ':\nValue:', eigenvalues[i], '\nVector:\n', eigenvectors[:,i],'\n\n')
plt.title('Eigenvectors and sample distribution')
plt.show()

In [ ]:
# Now the orthogonal factors are simply the eigenvectors times the raw factors...here I make up 'raw' factor loadings and compute 'orthogonalized' loadings
# Annoyingly you have to transpose the eigenvector matrix

#
# CHANGE NUMBERS HERE IF YOU WANT
raw_factor_loadings = np.array([1.,-1.])

# Construct 2 x 2 matrics to translate raw factors to orthogonal, and vice versa.
# The translation includes both shrinkage and rotation
raw_to_orth_matrix = (np.multiply(np.sqrt(eigenvalues),eigenvectors)).transpose()
orth_to_raw_matrix = np.linalg.inv(raw_to_orth_matrix)
print('Raw to Orthogonal Translation matrix:\n', raw_to_orth_matrix)
print('Orthogonal to Raw Translation matrix:\n', orth_to_raw_matrix)
orth_factor_loadings = np.squeeze(np.asarray(np.matmul(raw_to_orth_matrix, raw_factor_loadings)))
recomputed_raw_factor_loadings = np.squeeze(np.asarray(np.matmul(orth_to_raw_matrix, orth_factor_loadings.transpose()).transpose()))
print('Raw Factors : ', raw_factor_loadings, '--(orth-space)-->', orth_factor_loadings, '--(raw-space)-->', recomputed_raw_factor_loadings)
ax=plt.axes()
arrow_w_title(ax, raw_factor_loadings[0], raw_factor_loadings[1], 'Raw Factor')
arrow_w_title(ax, orth_factor_loadings[0], orth_factor_loadings[1], 'Orthogonal Factor')
ax.set_xlim(-3,3)
ax.set_ylim(-2,2)
plt.show()



In [ ]:
colors = ['r','g','b', 'y']
ax=plt.axes()
for i in range(len(colors)):
  vec=np.array(np.random.uniform(-1,1, size=[2]))
  orth_vec = np.squeeze(np.asarray(np.matmul(raw_to_orth_matrix,vec)))
  arrow_w_title(ax, vec[0], vec[1], '', colors[i], head_length=0.0)
  arrow_w_title(ax, orth_vec[0], orth_vec[1], '', colors[i], head_length=0.0)
  arrow_w_title(ax, orth_vec[0]-vec[0], orth_vec[1]-vec[1], '', colors[i], x0 = vec[0], y0 = vec[1], head_length=0.2, head_width=0.2)

ax.set_xlim(-2.5,2.5)
ax.set_ylim(-2.5,2.5)
plt.title('Vector translations from raw to orthogonal...no arrow means post translation')
plt.show()


In [ ]:
def compute_variance_orth_factors(orth_loadings):
  raw_loadings = np.matmul(orth_to_raw_matrix, orth_loadings.transpose())
  return compute_variance(raw_loadings.transpose())
def compute_variance_orth_factors_xy(x,y):
  return compute_variance_orth_factors(np.array([x,y]))

#
# Plot iso-risk curves in transformed space
x, y = np.meshgrid(np.linspace(-5, 5, 50), np.linspace(-5, 5, 50))
fig, ax = plt.subplots(1)
v_func_orth = np.vectorize(compute_variance_orth_factors_xy)
ax.contour(x, y, v_func_orth(x, y), [1., 2., 3., 4., 5.])
ax.grid()
plt.title( 'Equi-variance contour plot')
plt.xlabel('Orthogonal Factor 1 Exposure')
plt.ylabel('Orthogonal Factor 2 Exposure')
plt.show()


In [ ]:
#
# The following show variance at various points on orth-space.
print('The following shows that, in orthogonal space, variaince is coordinate-invariate and orthogonal')
for coords in [ [1,0], [0,1],[-1,0],[0,-1], [1,1], [-1,-1], [1,-1], [-1,1]]:
  print('Variance at {}: In orth-space {:.6f}, in raw space {:.6f}'.format(coords, compute_variance_orth_factors_xy(coords[0],coords[1]), compute_variance_xy(coords[0],coords[1])))
  

In [ ]:
orth_samples = np.matmul(samples, eigenvectors)
orth_samples = np.multiply(orth_samples, np.sqrt(eigenvalues))

# Use .A1 to get as an array
plt.scatter(orth_samples[:,0].A1,orth_samples[:,1].A1)
plt.title('Distribution in orthogonal space...note that the axes may be skewed')
plt.show()


In [ ]:
compute_variance_xy(-.423, 1.577)